## The first solution to assignment 3 using spacy

In [19]:
import os
import re
import random
import string
from typing import List
import spacy
from spacy import displacy
from spacy.util import minibatch, compounding
from spacy.training import Example
import json
from spacy.pipeline.spancat import DEFAULT_SPANCAT_MODEL
import pandas as pd
from tqdm import tqdm

nlp = spacy.blank('ru')
span_key = "sc"

In [20]:
%%capture
from datasets import load_dataset

In [21]:
# loading data
with open("../../data/train.jsonl", "r") as f:
    train_data = [json.loads(line) for line in f.readlines()]

with open("../../data/dev.jsonl", "r") as f:
    dev_data = [json.loads(line) for line in f.readlines()]

Here we will prepare data for training. During training, spacy requires Example objects with the full text and annotations of spans and labels. Lets create them

In [24]:
train_data_examples = []
counter = 0 # counter of skipped samples for some reasons

for sample in train_data:
  id = sample["id"]
  text = sample["sentences"]
  entities = sample["ners"][:2]

  # add 1 to every second value in every tuple to make second value exclusive
  entities = [[tup[0], tup[1]+1, tup[2]] for tup in entities]

  # This is the argument Example object needs when creating
  annotation = {'spans' : {span_key : entities}}
  try:

    # create Example from text and annotations (with spans and labels)
    train_data_examples.append(Example.from_dict(nlp.make_doc(text), annotation))
  except:
    # If we could not create Example for this text, then skip. This happens if boundaries of words are not spaces or punctuation
    counter +=1
    continue
counter

2

In [26]:
%%capture
entities = load_dataset('MalakhovIlya/RuNNE', 'ent_types')['ent_types']

In [27]:
config = {
    # this refers to the minimum probability to consider a prediction positive
    "threshold": 0.85,
    # the span key refers to the key in doc.spans
    "spans_key": span_key,
    # this refers to the maximum number of labels to consider positive per span
    "max_positive": None,
    # a model instance that is given a list of documents with start end indices representing the labelled spans
    "model": DEFAULT_SPANCAT_MODEL,
    # A function that suggests spans. This suggester is fixed n-gram length of up to 5 tokens
    "suggester": {"@misc": "spacy.ngram_suggester.v1", "sizes": [1, 2, 3, 4, 5]},
}

#add spancat component to nlp object
nlp.add_pipe("spancat", config=config)

#get spancat component
span=nlp.get_pipe('spancat')

#Add labels to spancat component
for label in entities["type"]:
    span.add_label(label)

In [28]:
pipe_exceptions = ["spancat"]

# disable redundant pipelines
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# initialise spacy object
nlp.initialize()
sgd = nlp.create_optimizer()

Next is training process

In [34]:
with nlp.disable_pipes(*unaffected_pipes):
    for iteration in tqdm(range(50)):
        random.shuffle(train_data_examples)
        losses = {}
        batches = minibatch(train_data_examples, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(list(batch), losses=losses, drop=0.1, sgd=sgd)

        print(f"iteration {iteration} train loss: {losses['spancat']:.2f}")

  2%|▏         | 1/50 [00:32<26:30, 32.47s/it]

iteration 0 train loss: 1018.34


  4%|▍         | 2/50 [01:04<25:32, 31.93s/it]

iteration 1 train loss: 1018.05


  6%|▌         | 3/50 [01:35<24:49, 31.70s/it]

...

iteration 47 train loss: 586.13


 98%|█████████▊| 49/50 [26:06<00:31, 31.22s/it]

iteration 48 train loss: 583.00


100%|██████████| 50/50 [26:44<00:00, 32.10s/it]

iteration 49 train loss: 572.41


Having the model, we can pass the test/dev data to it to get the result

In [35]:
with open("test.jsonl", "w") as f:
    for sample in dev_data:
        text = sample["senences"]
        id = sample["id"]
        doc = nlp(text)
        spans = doc.spans[span_key]
        d = {"id" : id, "ners" : []}
        for span, confidence in zip(spans, spans.attrs["scores"]):
            d["ners"].append([span.start, span.end, span.label_])
        f.write(str(d).replace("'", "\""))
        f.write("\n")

In [36]:
!zip test test.jsonl

updating: test.jsonl (164 bytes security) (deflated 84%)


In [1]:
import shutil

In [2]:
shutil.move("./test.jsonl", "./output/test.jsonl")
shutil.move("./test.zip", "./output/test.zip")

'./output/test.zip'